In [ ]:
import keras
from keras.layers import Conv2D, DepthwiseConv2D, BatchNormalization, ReLU, Add, Input, GlobalAveragePooling2D, Dense
from keras.models import Model

import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def inverted_residual_block(x, expansion_factor, filters, stride):
    in_channels = x.shape[-1]
    x_shortcut = x

    # Expansion
    x = Conv2D(expansion_factor * in_channels, kernel_size=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU(max_value=6)(x)

    # Depthwise Convolution
    x = DepthwiseConv2D(kernel_size=3, strides=stride, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU(max_value=6)(x)

    # Projection
    x = Conv2D(filters, kernel_size=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    # Skip connection
    if in_channels == filters and stride == 1:
        x = Add()([x_shortcut, x])

    return x

In [ ]:
def MobileNetV2(input_shape=(224, 224, 3), num_classes=2):
    img_input = Input(shape=input_shape)

    # Initial Conv layer
    x = Conv2D(32, kernel_size=3, strides=(2, 2), padding='same', use_bias=False)(img_input)
    x = BatchNormalization(name='bn_Conv1')(x)
    x = ReLU(max_value=6, name='Conv1_relu')(x)

    # Inverted Residual blocks
    x = inverted_residual_block(x, expansion_factor=1, filters=16, stride=1)

    x = inverted_residual_block(x, expansion_factor=6, filters=24, stride=2)
    x = inverted_residual_block(x, expansion_factor=6, filters=24, stride=1)

    x = inverted_residual_block(x, expansion_factor=6, filters=32, stride=2)
    x = inverted_residual_block(x, expansion_factor=6, filters=32, stride=1)
    x = inverted_residual_block(x, expansion_factor=6, filters=32, stride=1)

    x = inverted_residual_block(x, expansion_factor=6, filters=64, stride=2)
    x = inverted_residual_block(x, expansion_factor=6, filters=64, stride=1)
    x = inverted_residual_block(x, expansion_factor=6, filters=64, stride=1)
    x = inverted_residual_block(x, expansion_factor=6, filters=64, stride=1)

    x = inverted_residual_block(x, expansion_factor=6, filters=96, stride=1)
    x = inverted_residual_block(x, expansion_factor=6, filters=96, stride=1)
    x = inverted_residual_block(x, expansion_factor=6, filters=96, stride=1)

    x = inverted_residual_block(x, expansion_factor=6, filters=160, stride=2)
    x = inverted_residual_block(x, expansion_factor=6, filters=160, stride=1)
    x = inverted_residual_block(x, expansion_factor=6, filters=160, stride=1)

    x = inverted_residual_block(x, expansion_factor=6, filters=320, stride=1)

    # Final Conv layer
    x = Conv2D(1280, kernel_size=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU(max_value=6)(x)

    # Global Average Pooling
    x = GlobalAveragePooling2D()(x)

    # Output layer
    x = Dense(num_classes, activation='softmax')(x)

    # Create model
    model = Model(inputs=img_input, outputs=x)

    return model

model = MobileNetV2()

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['conv2d[0][0]']              
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [ ]:
train_dir = '/content/drive/MyDrive/Data_Int/BreakHis_v1_100/output/train'
validation_dir = '/content/drive/MyDrive/Data_Int/BreakHis_v1_100/output/val'
test_dir = '/content/drive/MyDrive/Data_Int/BreakHis_v1_100/output/test'


img_size = (224, 224)
batch_size = 32


train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 1455 images belonging to 2 classes.
Found 415 images belonging to 2 classes.
Found 211 images belonging to 2 classes.


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    epochs=55,
    validation_data=validation_generator
)

Epoch 1/55
46/46 [==============================] - 46s 1s/step - loss: 0.4001 - accuracy: 0.8371 - val_loss: 0.9618 - val_accuracy: 0.6916
Epoch 2/55
46/46 [==============================] - 48s 1s/step - loss: 0.4033 - accuracy: 0.8151 - val_loss: 1.1667 - val_accuracy: 0.6916
Epoch 3/55
46/46 [==============================] - 48s 1s/step - loss: 0.3561 - accuracy: 0.8488 - val_loss: 1.0472 - val_accuracy: 0.6916
Epoch 4/55
46/46 [==============================] - 49s 1s/step - loss: 0.3336 - accuracy: 0.8577 - val_loss: 0.8578 - val_accuracy: 0.6916
Epoch 5/55
46/46 [==============================] - 48s 1s/step - loss: 0.3112 - accuracy: 0.8550 - val_loss: 1.2944 - val_accuracy: 0.6916
Epoch 6/55
46/46 [==============================] - 47s 1s/step - loss: 0.2986 - accuracy: 0.8674 - val_loss: 1.0297 - val_accuracy: 0.6916
Epoch 7/55
46/46 [==============================] - 49s 1s/step - loss: 0.3226 - accuracy: 0.8639 - val_loss: 1.2772 - val_accuracy: 0.6916
Epoch 8/55
46/46 [==

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

7/7 [==============================] - 77s 13s/step - loss: 2.5667 - accuracy: 0.5450
Test accuracy: 54.50%


In [ ]:
model.save('/content/drive/My Drive/path_to_save_model/mobilenet_v2_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
